In [0]:
# Entropy Minimization on the cifar10 dataset
# Loss functions implemented in this code is as per the concepts introduced by Professor Dr. Hemanth Kumar Demakethepalli Venkateswara 
#in the course CSE 569: Fundamentals Of Statistical Learning at Arizona State University, Tempe, AZ.
# Also code from below mentioned web site is used as a reference to build model.
#https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

#@author: Jayesh Kudase

import sys
import numpy as np
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as k
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix

labeled_size=5000
unlabeled_size=45000
# labeled_size=25000
# unlabeled_size=25000
# labeled_size=15000
# unlabeled_size=35000

# load train and test dataset
def load_dataset():
  # load dataset
  (trainX, trainY), (testX, testY1) = cifar10.load_data()
  real_trainX,real_trainY=trainX[:labeled_size,:],trainY[:labeled_size,:]
  unlab_trainX, unlab_trainY=trainX[:labeled_size],trainY[:labeled_size]
  
  unlab_trainY_one_hot=np.ndarray(shape=(unlabeled_size,10));
  for i in range (unlab_trainY_one_hot.shape[0]):
    for j in range (unlab_trainY_one_hot.shape[1]):
      unlab_trainY_one_hot[i,:]=1
  
  # one hot encode target values
  trainY = to_categorical(real_trainY)
  
  print(trainY.shape)
  print(unlab_trainY_one_hot.shape)
  after_trainY=np.concatenate((trainY,unlab_trainY_one_hot),axis=0)
  
  testY = to_categorical(testY1)
  return trainX, after_trainY, testX, testY, testY1

# scale pixels
def prep_pixels(train, test):
  # convert from integers to floats
  train_norm = train.astype('float32')
  test_norm = test.astype('float32')
  # normalize to range 0-1
  train_norm = train_norm / 255.0
  test_norm = test_norm / 255.0
  # return normalized images
  return train_norm, test_norm

# define custom loss

def custom_loss(yTrue,yPred):
  lab_loss=k.sum(yTrue[:labeled_size]*k.log(yPred[:labeled_size]))/(-labeled_size)
  unlab_loss=k.sum(yPred[labeled_size:]*k.log(yPred[labeled_size:]))/(-unlabeled_size)
  return lab_loss+unlab_loss

# define cnn modez
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.001)))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss=custom_loss, metrics=['accuracy'])
 
  return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
  # plot loss
  pyplot.subplot(211)
  pyplot.title('Cross Entropy Loss')
  pyplot.plot(history.history['loss'], color='blue', label='train')
  pyplot.plot(history.history['val_loss'], color='orange', label='test')
  pyplot.xlabel('epoch')
  pyplot.ylabel('Loss')
  pyplot.legend()
  pyplot.figure()
  # plot accuracy
  pyplot.subplot(212)
  pyplot.title('Classification Accuracy')
  pyplot.plot(history.history['acc'], color='blue', label='train')
  pyplot.plot(history.history['val_acc'], color='orange', label='test')
  pyplot.xlabel('epoch')
  pyplot.ylabel('Accuracy')
  pyplot.legend()
  pyplot.figure()
  # save plot to file
  filename = sys.argv[0].split('/')[-1]
  pyplot.savefig(filename + '_plot.png')
  pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
  # load dataset
  trainX, trainY, testX, testY, testY1 = load_dataset()
  # prepare pixel data
  trainX, testX = prep_pixels(trainX, testX)
  # define model
  model = define_model()
  # fit model
  history = model.fit(trainX, trainY, epochs=100, batch_size=500, validation_data=(testX, testY), verbose=1)
  #confusion matrix
  # y_pred_ohe = model.predict(testX) 
  # y_pred_labels = np.argmax(y_pred_ohe, axis=1) 
  # confusion_matrix = metrics.confusion_matrix(y_true=testY1, y_pred=y_pred_labels)
  # pyplot.imshow(confusion_matrix)
  # evaluate model
  _, acc = model.evaluate(testX, testY, verbose=0)
  print('> %.3f' % (acc * 100.0))
  # learning curves
  summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

(5000, 10)
(45000, 10)
Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 10s 201us/step - loss: 3.2859 - acc: 0.0975 - val_loss: 1.3772 - val_acc: 0.0896
Epoch 2/100
50000/50000 [==============================] - 9s 188us/step - loss: 3.2418 - acc: 0.0946 - val_loss: 1.3723 - val_acc: 0.0957
Epoch 3/100
50000/50000 [==============================] - 9s 188us/step - loss: 3.2366 - acc: 0.1003 - val_loss: 1.3675 - val_acc: 0.0995
Epoch 4/100
50000/50000 [==============================] - 9s 188us/step - loss: 3.2317 - acc: 0.1028 - val_loss: 1.3628 - val_acc: 0.1051
Epoch 5/100
50000/50000 [==============================] - 9s 188us/step - loss: 3.2269 - acc: 0.1090 - val_loss: 1.3581 - val_acc: 0.1098
Epoch 6/100
50000/50000 [==============================] - 9s 188us/step - loss: 3.2222 - acc: 0.1095 - val_loss: 1.3536 - val_acc: 0.1133
Epoch 7/100
50000/50000 [==============================] - 9s 188us/step - loss: 3.2175 - ac